In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tf
from skimage import measure, morphology, segmentation, filters
from scipy import ndimage


def load_image(image_path):
    with tf.TiffFile(image_path) as tif:
        return tif.pages[0].asarray()


def create_binary_mask(channel, threshold=10000):
    return np.where(channel > threshold, 255, 0).astype(np.uint8)


def apply_watershed(binary_mask):
    distance = ndimage.distance_transform_edt(binary_mask)
    local_maxi = morphology.local_maxima(distance)
    markers = measure.label(local_maxi)
    return segmentation.watershed(-distance, markers, mask=binary_mask)


def detect_edges(watershed_mask):
    edges = filters.sobel(watershed_mask)
    return (edges > 0).astype(np.uint8) * 255


def combine_masks(binary_mask, edges_binary):
    return np.where(edges_binary > 0, 0, binary_mask)


def remove_small_objects(mask, min_area=100):
    return morphology.remove_small_objects(mask.astype(bool), min_size=min_area)


def main():
    # File paths
    image_path = "j.maragall/Replication2 _DAPImapping/19-4.tif"
    cleaned_mask_path = "R2_DAPI_cleaned_mask.tif"

    # Load image
    first_channel = load_image(image_path)

    # Process image
    binary_mask = create_binary_mask(first_channel)
    watershed_mask = apply_watershed(binary_mask)
    edges_binary = detect_edges(watershed_mask)
    final_mask = combine_masks(binary_mask, edges_binary)
    cleaned_mask = remove_small_objects(final_mask)

    # Save cleaned mask
    tf.imsave(cleaned_mask_path, cleaned_mask.astype(np.uint8) * 255)

    # Define a region of interest (x1, x2, y1, y2)
    x1, x2, y1, y2 = 1000, 2000, 1000, 2000

    # Crop the cleaned mask to the region of interest
    roi = cleaned_mask[y1:y2, x1:x2]

    # Visualize the region of interest of the cleaned mask
    plt.imshow(roi, cmap='gray')
    plt.axis('off')
    plt.title('Region of Interest in Cleaned Mask')
    plt.show()


if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: '/blue/pinaki.sarder/j.maragall/CellMapping/19-4.tif'